In [21]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Fri Dec 23 11:33:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:0B:00.0  On |                  N/A |
| 30

In [ ]:
# from transformers import BertTokenizer, BertModel
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [2]:
from transformers import (AutoConfig, AutoModel, AutoTokenizer)
config = AutoConfig.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
special_labels = {',': 'I-COMMA',
                  '.': 'I-DOT',
                  '?': 'I-QMARK',
                  '!': 'I-EMARK',
                  ':': 'I-COLON',
                  ';': 'I-SEMICOLON'}
normal_label = 'O'

In [18]:
def descrete_and_label(list_of_lines):
    list_of_lists = []
    for i, line in enumerate(list_of_lines):
        tkn_line = tokenizer.tokenize(line)
        if len(tkn_line) < 10 or len(tkn_line) > 510:
            continue
        for word in line.split():
            lbl = normal_label
            brek = False
            sl = special_labels.get(word, None)
            if sl:
                if list_of_lists:
                    list_of_lists[-1][2] = sl
                    brek = True
            if not brek:
                list_of_lists.append([i, word, lbl])
    return list_of_lists

In [12]:
from datasets import load_dataset, ReadInstruction

In [13]:
dataset = load_dataset('wikitext', 'wikitext-103-v1')

Found cached dataset wikitext (/home/mostafa/.cache/huggingface/datasets/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


  0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
print(len(dataset['train']), len(dataset['validation']), len(dataset['test']))

1801350 3760 4358


In [15]:
print(dataset['train'][100602]['text'])
my_line = dataset['train'][100602]['text']

 The Spring Creek Powerplant is located at the base of the Spring Creek Dam , and is actually supplied by flow from Whiskeytown Lake . The lake , formed by Whiskeytown Dam , is on Clear Creek , a drainage downstream along the Sacramento River from Spring Creek . Both streams run east into the river . The power plant was completed and began operations in 1964 , with a capacity of 150 @,@ 000 kW . The capacity was later upgraded to 180 @,@ 000 kW . The plant is a peaking power plant with two separate generators , generating power for operations and distributing excess power to the local power grid . Water from Whiskeytown Lake is diverted through the Spring Creek Tunnel , a conduit roughly 2 @.@ 4 miles ( 3 @.@ 9 km ) long and 18 @.@ 5 feet ( 5 @.@ 6 m ) in diameter . 



In [16]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", output_hidden_states=True)

In [17]:
tokenizer.tokenize(my_line)

['the',
 'spring',
 'creek',
 'powerplant',
 'is',
 'located',
 'at',
 'the',
 'base',
 'of',
 'the',
 'spring',
 'creek',
 'dam',
 ',',
 'and',
 'is',
 'actually',
 'supplied',
 'by',
 'flow',
 'from',
 'whiskey',
 '##town',
 'lake',
 '.',
 'the',
 'lake',
 ',',
 'formed',
 'by',
 'whiskey',
 '##town',
 'dam',
 ',',
 'is',
 'on',
 'clear',
 'creek',
 ',',
 'a',
 'drainage',
 'downstream',
 'along',
 'the',
 'sacramento',
 'river',
 'from',
 'spring',
 'creek',
 '.',
 'both',
 'streams',
 'run',
 'east',
 'into',
 'the',
 'river',
 '.',
 'the',
 'power',
 'plant',
 'was',
 'completed',
 'and',
 'began',
 'operations',
 'in',
 '1964',
 ',',
 'with',
 'a',
 'capacity',
 'of',
 '150',
 '@',
 ',',
 '@',
 '000',
 'kw',
 '.',
 'the',
 'capacity',
 'was',
 'later',
 'upgraded',
 'to',
 '180',
 '@',
 ',',
 '@',
 '000',
 'kw',
 '.',
 'the',
 'plant',
 'is',
 'a',
 'peaking',
 'power',
 'plant',
 'with',
 'two',
 'separate',
 'generators',
 ',',
 'generating',
 'power',
 'for',
 'operations',
 '

In [19]:
import pandas as pd

def save_dataset(ds, path):
    filtered = []
    filtered += [i['text'] for i in ds if len(i['text']) > 20]
    dataset_1 = descrete_and_label(filtered)
    train_data = pd.DataFrame(dataset_1, columns=["sentence_id", "words", "labels"])
    train_data.to_csv(path, index=False)

In [20]:
binz = 10
for i in range(binz):
  print(i * (100/binz), (i+1) * (100/binz))
  sub_dataset = load_dataset('wikitext', 'wikitext-103-v1', split=ReadInstruction(
      'train', from_=int(i * (100/binz)), to=int((i+1) * (100/binz)), unit='%', rounding='pct1_dropremainder'))
  print(len(sub_dataset))
  save_dataset(sub_dataset, f'./preprocessed_wikitext/train{i}-{binz}.csv')

0.0 10.0


Found cached dataset wikitext (/home/mostafa/.cache/huggingface/datasets/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


180130


Token indices sequence length is longer than the specified maximum sequence length for this model (647 > 512). Running this sequence through the model will result in indexing errors


10.0 20.0


...

180130
80.0 90.0


Found cached dataset wikitext (/home/mostafa/.cache/huggingface/datasets/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


180130
90.0 100.0


Found cached dataset wikitext (/home/mostafa/.cache/huggingface/datasets/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


180130


In [21]:
validation = load_dataset('wikitext', 'wikitext-103-v1', split='validation')
test = load_dataset('wikitext', 'wikitext-103-v1', split='test')

Found cached dataset wikitext (/home/mostafa/.cache/huggingface/datasets/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
Found cached dataset wikitext (/home/mostafa/.cache/huggingface/datasets/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


In [22]:
len(validation), len(test)

(3760, 4358)

In [23]:
save_dataset(validation, f'./preprocessed_wikitext/validation.csv')

In [24]:
save_dataset(test, f'./preprocessed_wikitext/test.csv')